In [18]:
import pandas as pd
from artifacts.ai_pc import main_proc_change
from config.params import ProcChangeConfig
import csv
from datetime import datetime
df = pd.read_csv('AAM_MEXICO_DATA_ACCELERATION_EMA2252M10022.csv')

In [19]:
# Creating Batch Using MEASUREMENT_DATE(1)
# Define a function to convert datetime string to desired format
def convert_to_custom_format(datetime_str):
    datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    return datetime_obj.strftime('%Y%m%d%H%M%S')

# Fetching the Data in 8 hour Period Based as similar to what described in the fetching schema
# Set the timestamp as the DataFrame index
df['MEASUREMENT_DATE'] = pd.to_datetime(df['MEASUREMENT_DATE'],format ='%Y%m%d%H%M%S')
df.set_index('MEASUREMENT_DATE',inplace=True)
# filename = filePath.split("/")[-1].split(".")[0]
# id = filename
# Resample the data on an hourly basis and store only the non-empty hourly frames in a list
hourly_frames = []
hourlyshotsLabel = []
hourname = []
for name, group in df.resample("H"):
    if not group.empty:
        hourname.append(name)
        hourly_frames.append(group)
first_batch = hourly_frames[9:17]
df_first_batch = pd.concat(first_batch)

df.reset_index(drop = False,inplace=True)
df_first_batch.reset_index(drop = False,inplace=True)

df_first_batch['MEASUREMENT_DATE'] = df_first_batch['MEASUREMENT_DATE'].astype(str)
df_first_batch
# Apply the function to the entire column
df_first_batch['MEASUREMENT_DATE'] = df_first_batch['MEASUREMENT_DATE'].apply(convert_to_custom_format)
df = df_first_batch
# print(df.head())
# logger.info("DataFrame:\n%s", df)
# Convert DataFrame to the "fetchScheme Result" format
# fetch_scheme_result = df.to_dict(orient='records')

# logger.info('fetchScheme Result : %s', fetch_scheme_result)
# Print the result
# print()
# for row in fetch_scheme_result:
#     # print(row)
#     pass

# Creating Batch Using MEASUREMENT_DATE(3)
try: 
    dummy_fetchResult = {
        "data": {
            "moldId": int(df['MOLD_ID'].unique()[0]),
            "counterId": str(df['COUNTER_ID'].unique()[0]),
            "startHour":str(df['MEASUREMENT_DATE'][0])[:-4], # Left side of the Inference Boundary
            "contractedCycleTime": 360,
            "accelerations": df['ACCELERATIONS'].tolist(),
            "dataAccId": df['ID'].tolist(),
            "measurementDate": df['MEASUREMENT_DATE'].tolist(),
            "procChanged": [None] * len(df['ID'].tolist()),  # Example process change indicators
            "similarityMetric": [None] * len(df['ID'].tolist()),
            "similarityMetricHr": [None] * len(df['ID'].tolist()),
        }
    }
except Exception as e:
    print(e)
    dummy_fetchResult = {
    "data": {
        "moldId": df['TERMINAL_ID'].unique()[0],
        "counterId": str(df['COUNTER_ID'].unique()[0]),
        "startHour":str(df['MEASUREMENT_DATE'][0])[:-4], # Left side of the Inference Boundary
        "contractedCycleTime": 360,
        "accelerations": df['ACCELERATIONS'].tolist(),
        "dataAccId": df['ID'].tolist(),
        "measurementDate": df['MEASUREMENT_DATE'].tolist(),
        "procChanged": [None] * len(df['ID'].tolist()),  # Example process change indicators
        "similarityMetric": [None] * len(df['ID'].tolist()),
        "similarityMetricHr": [None] * len(df['ID'].tolist()),
    }
    }
batch_start_hour=str(df['MEASUREMENT_DATE'][0])[:-4]

cannot convert float NaN to integer


In [20]:
from artifacts.ai_pc import parse_individual_records
from config.params import ProcChangeConfig
x=parse_individual_records(dummy_fetchResult
                           ,FIRST_SECTION_PROP=ProcChangeConfig.FIRST_SECTION_PROP,
                           BIN_WIDTH_FIRST_SECTION=ProcChangeConfig.BIN_WIDTH_FIRST_SECTION,
                           BIN_WIDTH_SECOND_SECTION=ProcChangeConfig.BIN_WIDTH_SECOND_SECTION)
x

[{'id': 11600, 'measurement_time': '20230427015154', 'measurement_hour': '2023042701', 'acc_feature': [65.535, 65.53499999999998], 'sim_metric': None, 'sim_metric_hr': None, 'proc_changed': None},
 {'id': 11615, 'measurement_time': '20230427051905', 'measurement_hour': '2023042705', 'acc_feature': [65.53499999999998, 0, 0, 65.53499999999998, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65.535, 0, 0, 65.535, 65.535, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65.535, 0], 'sim_metric': None, 'sim_metric_hr': None, 'proc_changed': None},
 {'id': 11616, 'measurement_time': '20230427052433', 'measurement_hour': '2023042705', 'acc_feature': [65.53499999999998, 0, 0, 65.535, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65.535, 65.535, 0, 0, 65.535, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65.535, 0], 'sim_metric': None, 'sim_metric_hr': None, 'proc_changed': None},
 {'id': 11617, 'measurement_

In [21]:
from artifacts.utils import check_data_valid
check_data_valid(x)

AttributeError: 'list' object has no attribute 'groupby'